<h1><center>Laboratorio 10: Interpretabilidad 🤖</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Bartolucci
- Nombre de alumno 2: Pilar Nilo


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/DiegoBarto01/MDS7202-pili-barto.git)

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

## Temas a tratar

- Clasificación usando `XGBoost`.
- Métodos Agnósticos Globales de Interpretabilidad (`Partial Dependence Plot`, `Permutation Feature Importance`)
- Métodos Agnósticos Locales de Interpretabilidad (`Scoped Rules`, `SHAP`)

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar un pipeline de clasificación con `XGBoost`.
- Implementar modelos de interpretabilidad para explicar el funcionamiento del modelo de clasificación.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# 1. Problemas Clínicos del Dr. Simi

<p align="center">
  <img src="https://gantz.cl/wp-content/uploads/2020/01/79024136_2718114448239059_7240913062570491904_o.jpg" width="350">
</p>

El reconocido **Dr. Simi**, famoso vendedor de medicamentos en latinoamerica, debido a la creciente prevalencia de enfermedades crónicas, como la diabetes, decidió abrir una clínica especializada en el tratamiento de esta enfermedad en nuestro país.

La clínica del Doctor Simi se convirtió en un lugar donde los pacientes con diabetes podrían recibir atención médica integral. El personal médico estaba compuesto por especialistas en endocrinología, nutrición y enfermería, todos capacitados en el manejo de la diabetes.

Sin embargo él se ha dado cuenta que los tiempos han cambiado y gracias a las tecnologías es posible generar mejores predicciones en la diabetes conociendo el historial médico de las personas. Por esto, el doctor se ha colocado una meta de incluir modelos de machine learning dentro de sus clínicas, para ello le ha solicitado crear un modelo capaz de predecir/clasificar diabetes pero le rogó que el desarrollo del modelo tuviera un especial enfoque en la interpretabilidad de lo que hace su modelo.

Para que usted pueda entrenar el modelo, Dr. Simi le ha entregado un dataset de todos los clientes que fueron detectados con diabetes a lo largo de la historia de la clinica. Con ello, adjunta el historial médico de las personas en forma de datos tabulares para que usted pueda realizar fácilmente la clasificación.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_10'
except:
    print('Ignorando conexión drive-colab')

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(path + '/diabetes_data.csv')
df.head(5)

## 2. Clasificación de pacientes con diabetes (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/QH--g3ZaSbsAAAAC/dr-simi-abrazo.gif" width="400">
</p>

Tareas:
1. En primer lugar, el reconocido doctor le pide entrenar un modelo de `XGBoost` utilizando como target la columna `Diabetes` del dataset `diabetes_data.csv`. Para el entrenamiento, realice los siguientes pasos:
  * Realice una breve exploración de los datos y determine si aplicará transformaciones (MinMaxScaler, StandardScaler, etc.) en alguna/s de las variables. (1 punto)
  * Cree un conjunto de entrenamiento y uno de prueba, con una proporción de 1/3 en el conjunto de prueba. (0.5 puntos)
  * Cree un ColumnTransformer de preprocesamiento donde aplique las transformaciones determinadas anteriormente. Fije el parámetro `verbose_feature_names_out=False` y fije la salida del ColumnTransformer en formato pandas mediante el método `.set_output(transform='pandas')`. (1 punto)
  *  Cree un pipeline donde integre el preprocesamiento y el modelo `XGBoost` y entrene el modelo. Luego utilice `classification_report(..)` para reportar el desempeño del modelo. (1 punto)

Comente sus decisiones y los resultados obtenidos con el modelo.

2. Luego, le pide responder las siguientes preguntas:
  *  ¿Es acaso un buen predictor de diabetes? (0.5 puntos)
  * ¿Qué buscan explicar las métricas utilizadas? (0.5 puntos)
  * ¿Las métricas utilizadas para medir la predictibilidad le permiten asegurar que su modelo haga una buena elección de las features?(0.5 puntos)

In [ ]:
!pip install xgboost

In [ ]:
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

In [ ]:
def exploratory_data_analysis(dataframe):
  #1. dimension df
  print(f"1. La dimension del dataframe es: {dataframe.shape} \n")
  #2. nombres de las columnas del df
  print(f"2. Las columnas que conforman el dataframe son: \n {dataframe.columns} \n")
  #3. mostrar los primeros 5 elementos, 5 ultimos elementos y 5 elementos aleatoreos
  print("3.a Los 5 primeros elementos del dataframe son:")
  display(dataframe.head(5))
  print("3.b Los 5 últimos elementos del dataframe son:")
  display(dataframe.tail(5))
  print("3.c 5 elementos al azar del dataframe son:")
  display(dataframe.sample(5, random_state=10))
  #4. descripcion rapida de las variables numericas del dataframe
  print(f"4. Descripción de las variables numéricas del dataframe:")
  display(dataframe.describe())
  #5. mensaje con la cantidad de valores nulos
  print(f"5. La cantidad de valores nulos en el dataframe son: \n {dataframe.isna().sum()} \n")
  #6. mensaje que detalle la cantidad de valores unicos
  print(f"6. La cantidad de valores unicos en el dataframe son: \n {dataframe.nunique()} \n")


In [ ]:
exploratory_data_analysis(df)

In [ ]:
#edades unicas del df
df['Age'].unique()

In [ ]:
#1. entrenar un modelo de XGBoost utilizando como target la columna Diabetes del dataset diabetes_data.csv
#a. realizar exploracion de datos y determine si aplicara transformaicones de datos en alguna de las variables
for col in df.columns:
    fig = px.histogram(df, x=col, title=f'Visualización de la distribución de variable {col}') #ver si es solo histo o podria ser otro
    fig.show()


*1. transformación de columnas*

Se aplicará MinMaxScaler a las columnas MentHlth, PhysHlth y Age ya que no siguen una distribución normal.

Por otro lado, se aplicará StandardScaler en la columna BMI ya que sigue una distribución normal.



In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
var_minmax=['MentHlth','PhysHlth','Age']
var_standarscaler=['BMI']
var_onehot=['GenHlth']
passthrough=['Sex', 'HighChol', 'CholCheck', 'Smoker',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'DiffWalk', 'Stroke', 'HighBP']
#transformo los dato

X = df.drop(columns=['Diabetes'])
y = df['Diabetes']
#b.crear un conjunto de entrenamiento y uno de prueba, con una proporción de 1/3 en el conjunto de prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=444)
#c.Cree un ColumnTransformer de preprocesamiento donde aplique las transformaciones determinadas anteriormente. Fije el parámetro verbose_feature_names_out=False
#y fije la salida del ColumnTransformer en formato pandas mediante el método .set_output(transform='pandas')

preprocessor = ColumnTransformer(
                  [('Min Max Scaler', MinMaxScaler(), var_minmax),
                  ('standard Scaler', StandardScaler(), var_standarscaler),
                  ('one hot encoder',OneHotEncoder(sparse_output=False),var_onehot),
                  ('Passthrough', 'passthrough', passthrough)],
                  verbose_feature_names_out=False).set_output(transform='pandas')
#d.Cree un pipeline donde integre el preprocesamiento y el modelo XGBoost y entrene el modelo. Luego utilice classification_report(..)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


In [ ]:
#d.Cree un pipeline donde integre el preprocesamiento y el modelo XGBoost y entrene el modelo. Luego utilice classification_report(..)
print(classification_report(y_test, y_pred))

*2.preguntas*


*   ¿Es acaso un buen predictor de diabetes?

El modelo anterior no es un buen predictor de diabetes, si bien tiene un alto accuracy de 0.73, este resulta bajo para el contexto en el que se está evaluando ya que en asuntos de salud se requieren modelo con un accuracy por sobre el 0.9. Lo anterior dado que se trabaja con personas y las predicciones deben ser fiables y con un bajo porcentaje de falsos positivos/falsos negativos, métrica que tambien se encuentra baja observando el F1-Score.


Otro punto destacable es que el dataframe solo incluye personas de 1 a 13 años, por ende hay dos opciones: 1.se quiere ver el indice de diabetes infantil entre este rango de edades o 2.se quiere estudiar la diabetes para cualquier rango etario, pero está sesgado a estas edades.

*   ¿Qué buscan explicar las métricas utilizadas?

Las métricas utilizadas buscan explicar que tan bueno es el modelo clasificando correctamente a la gente que tiene diabetes y a la que no, tambien los falsos negativos y positivos.

*   ¿Las métricas utilizadas para medir la predictibilidad le permiten asegurar que su modelo haga una buena elección de las features?

No dado que el modelo es sesgado, además no hay manera de deducir que caracteristicas son mejores que otras solo con estas métricas.


## 3. Importancia de las features con XGBoost (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/5JAj5_IiagEAAAAd/dr-simi-dr-simi-dance.gif" width="400">
</p>

Tareas:
1. Para añadir el toque de interpretabilidad que Dr. Simi le pide, se le pide calcular la **importancia de las features** del modelo entrenado utilizando todos los métodos (*weight*, *cover*, *gain*) que posee xgboost usando `plot_importance`. `Hint:` Puede acceder a un paso de un pipeline por su nombre mediante el método `.named_steps[...]` (3 puntos)
2. ¿Los resultados obtenidos con los diferentes métodos son compatibles?, comente sus resultados y a que se debe la igualdad o desigualdad que ve en los resultados. (1 punto)
3. Finalmente, ¿las importancias obtenidas son suficientes para obtener la interpretabilidad de un modelo que utiliza árboles? ¿Qué debilidad presenta este método? (1 punto)

In [ ]:
from xgboost import plot_importance

In [ ]:
#calcular la importancia de las features del modelo entrenado utilizando todos los métodos (weight, cover, gain) que posee xgboost usando plot_importance
# Hint: Puede acceder a un paso de un pipeline por su nombre mediante el método .named_steps[...
plot_importance(pipeline.named_steps['classifier'], importance_type='weight')

In [ ]:
plot_importance(pipeline.named_steps['classifier'], importance_type='cover')

In [ ]:
plot_importance(pipeline.named_steps['classifier'], importance_type='gain')

*Pregunta 2: ¿Los resultados obtenidos con los diferentes métodos son compatibles?, comente sus resultados y a que se debe la igualdad o desigualdad que ve en los resultados.*

No son compatibles entre si ya que miden cosas diferentes y según eso es lo que se visualiza en los gráficos anteriores.

El método weight mide que tan frecuente se usa cada característica y las mejores caracteristicas son: BMI, Age, PhysHlth y MentHlth.

El método cover indica  que tan útil fue la caracteristica en las separaciones de los nodos, las mejores caracteristicas fueron: CholCheck, GenHlth 1.0, BMI, GenHlth 5.0, HvyAlcoholConsump, GenHlth 2.0, HighChol y Stroke.

El método gain es una medida de la importancia de una característica para la mejora en el desempeño del modelo, las mejores caracteristicas son: HighBP, GenHlth 1.0 y GenHlth 2.0.


*Pregunta 3: ¿las importancias obtenidas son suficientes para obtener la interpretabilidad de un modelo que utiliza árboles? ¿Qué debilidad presenta este método?*

No son suficientes ya que hay más factores que influyen en la toma de decisiones del árbol.

La debilidad que presenta este método es que no es posible que interactuen entre sí y por lo tanto no se puede determinar que caracteristicas son las mejores o no, ya que ciertas caracteristicas al interactuar con otras pueden aumentar de manera significativa la predicción correcta como tambien disminuirla.


## 4. Métodos Agnósticos Globales (10 puntos)

<p align="center">
  <img src="https://media.tenor.com/JcRHtjVuXN8AAAAC/dr-simi-farmacias-similares.gif" width="400">
</p>

Tareas:
1. Para mitigar los problemas encontrados en la sección anterior, Dr. Simi le pide implementar un **método de permutación** que le permita observar la importancia de las features. `Nota:`Tenga cuidado con el orden de las columnas de este método. `Hint:` Puede obtener los features del clasificador con su respectivo orden mediante el método `.get_booster().feature_names` (2 puntos)
2. Para que su modelo sea consistente, repita el proceso **30 veces** y verifique la desviación estándar de sus resultados (¿Qué señala esta?). (2 puntos)
3. Visualice los resultados de este método en un gráfico. (2 puntos)
4. Además, responda las siguientes preguntas:
  - ¿Cómo mide la importancia de las features su propuesta? (1 punto)
  - ¿Qué features tienen un mayor impacto en la salida del modelo?. Comente las 5 primeras, ¿tienen sentido? (1 punto)
  - ¿Cómo cambian sus conclusiones con respecto a las features importances del punto anterior? (1 punto)
  - Nombre y explique 3 ventajas y 3 desventajas del método implementado. (1 punto)

In [ ]:
# Inserte su código aquí
# Se implementa el método de permutación
from sklearn.inspection import permutation_importance
permutation_results = permutation_importance(pipeline, X_test, y_test, n_repeats=30, random_state=444)

In [ ]:
# Se visualizan los resultados con la desviación estandar
import matplotlib.pyplot as plt

importance_features = pd.DataFrame(permutation_results.importances, index=X.columns)
importances_mean = importance_features.mean(axis=1)

importances_sorted = importance_features.loc[importances_mean.sort_values(ascending=True).index]

plt.figure(figsize=(10, 5))
importances_sorted.T.boxplot(vert=False)
plt.xlabel('Error')
plt.ylabel('Features')
plt.title('Importancia de las features con 30 repeticiones')
plt.show()

Respuestas:

P2 - La desviación estándar indica la variabilidad de la importancia en cada feature, ya que no se espera que sea la mmisma en cada iteración que se realiza, debido a que las reglas de evaluación en cada árbol varía. Una alta desviación estándar indicaría que la importancia del feature varia considerablemente entre cada iteración; mientras que en el caso contrario, señalaría que el feature es consistentemente importante.

P4 -
* ¿Cómo mide la importancia de las features su propuesta?

Con lo desarrollado, se mide la importancia a través del error obtenido en cada iteración evaluando en cada una de estas si el feature tiene una gran importancia en el modelo o no, a través de cambiar su valor individualmente.

* ¿Qué features tienen un mayor impacto en la salida del modelo?. Comente las 5 primeras, ¿tienen sentido?

Las features que tienen un mayor impacto en la salida del modelo son GenHlth, BMI, Age, HighBP y HighCol. Estas tienen sentido que sean las más altas ya que son factores que son determinantes en la presencia de Diabetes o no como el estado de salud, el peso, la edad, etc.

*  ¿Cómo cambian sus conclusiones con respecto a las features importances del punto anterior?

Las features más importantes, tanto en el punto anterior como en este, se mantienen por lo que se podría concluir que este método tiene relación con el método anterior en algún aspecto.

*  Nombre y explique 3 ventajas y 3 desventajas del método implementado.

Ventajas:

1.   Independencia del modelo interno: No depende de cómo el modelo está construido internamente, por lo que es generalizable con otros tipos.
2.   Fácil interpretración: Porporciona una métrica que se puede interpretar fácilmente, permitiendo una comunicación de resultados más fluida.
3.   Evaluación del impacto en el modelo: Se mide cómo afecta la permutación de cada feature en el rendimiento del modelo, obteniendo una idea clara de la importancia de cada una.

Desventajas:
1.   Alto costo computacional: Si se llega a requerir evaluar varias veces un modelo, es costoso en términos de tiempo y recursos computacionales.
2.   Sensibilidad ante variaciones: La importancia en las features es sensible a la variación generando distintos tiempos computacionales dependiendo de la variable modificada.
3.   Limitación ante complejidad: Probablemente no capture adecuadamente la importancia de las features en modelos con una alta complejidad.


## 5. Métodos Agnósticos Locales (20 puntos)

<p align="center">
  <img src="https://i.makeagif.com/media/10-24-2024/oMCrLI.gif" width="400">
</p>

### 5.1 Calculando Shap Values (4 puntos)

Tareas:
1. Alegre por saber cómo funciona el modelo de predicción a nivel general, Dr. Simi le pide ahora interpretar las predicciones de su modelo a nivel de paciente (es decir, desde un punto de vista **local**). Para esto, el ilustre farmacéutico le pide calcular los *shap values* de su modelo. (2 puntos)
2. ¿Qué representa cada número en su resultado? (1 punto)
3. ¿Es posible atribuir un significado a la positividad/negatividad de cada valor? (1 punto)

In [ ]:
!pip install shap

In [ ]:
import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")

In [ ]:
# Inserte código para calcular shapley values aquí
import shap
shap_explainer = shap.Explainer(pipeline.named_steps['classifier'],preprocessor.transform(X_test))
shap_values = shap_explainer(preprocessor.transform(X_test))

In [ ]:
shap_values

Respuestas:

P2 - Se presentan 3 tipos de valores shap, los valores.values, los cuales corresponden a el shap calculado para cada feature demostrando la importancia de cada uno de estos, los valores.base_values, los que corresponden a los valores base para las predicciones del modelo, y los valores.data, los que corresponden a el conjunto de datos sobre el cual se están calculando los valores shap.

P3 - Si, los valores positivos corresponden a que ese feature tiene una mayor importancia para el modelo, ya que este permitiría una predicción más exacta, mientras que los valores negativos indicarían que la presencia de este feature tiene un efecto negativo en la predicción causando que esta disminuya.

### 5.2 Aporte local (4 puntos)

1. Usando los *shap values* calculados, grafique el **aporte local** de las diferentes variables para las instancias **1**, **9** y **150** (1 punto).

2. Interprete sus resultados y responda:

  - ¿Qué variables afectan de manera positiva/negativa a la probabilidad de poseer diabetes? (1 punto)

  - ¿Existe algún patrón común entre las instancias analizadas? (1 punto)

  - ¿Es posible generalizar estas conclusiones a todo el dataset? (1 punto)

In [ ]:
# Inserte código para generar gráficos de aporte local aquí
# Variable 1
shap.summary_plot(shap_values[[1]], feature_names=preprocessor.transform(X_test).columns)

In [ ]:
# Variable 9
shap.summary_plot(shap_values[[9]], feature_names=preprocessor.transform(X_test).columns)

In [ ]:
# Variable 150
shap.summary_plot(shap_values[[150]], feature_names=preprocessor.transform(X_test).columns)

Respuestas:

a) Se puede observar que para los distintos casos varía el impacto que tienen los features, por ejemplo para el caso 1 y 150 HighBP implica de manera positiva, mientras que el caso 9 implica de manera negativa. Pero a grandes rasgos se conlcuye que Age implica de manera negativa igual que GenHlth_2.0 y BMI, GenHlth_5.0 y HighBP de manera positiva.

b) No se aprecia un patrón en común entre las instancias, sin embargo como se mencionó antes, hay algunos features que mantienen su positividad o negatividad en más de una instancia.

c) A partir de las instancias revisadas no se podrían generalizar para el dataset completo, ya que los resultados evaluados no presentan un patrón para aplicar.

### 5.3 Aporte global (4 puntos)

Genere ahora una visualización donde se grafique el aporte de cada feature a nivel **global** e interprete sus resultados. ¿Qué diferencias existen con las conclusiones generadas a nivel de instancia?

In [ ]:
# Inserte código para generar gráficos de aporte global aquí
shap.summary_plot(shap_values, feature_names=preprocessor.transform(X_test).columns)

Respuesta:

Se puede observar un impacto más general por feature a diferencia de a nivel de instancia que presenta puntos específicos que pueden o no ser represetativos para ese feature.

### 5.4 Scatter plot (4 puntos)


Grafique ahora un *scatterplot* entre los *shap values* y las primeras 5 features con mayor impacto global (un gráfico por cada feature), coloreando cada punto por la probabilidad de tener diabetes. ¿Qué puede concluir de sus resultados?

In [ ]:
# Inserte código para generar gráficos de feature vs shap value aquí
top5 = ['HighBP', 'BMI', 'Age', 'GenHlth_2.0', 'HighChol']
for feature in top5:
  shap.plots.scatter(shap_values[:, preprocessor.transform(X_test).columns.get_loc(feature)], color=shap_values)
  plt.show()

Respuestas:

En base a los resultados observados se puede observar que cada uno impacta de manera distinta la probabilidad de presentar Diabetes. Se puede observar que a mayor Age, un mayor HighChol, mayor HighBP y mayor BMI la probabilidad de tener Diabetes aumenta, mientras que si aumenta GenHlth_2.0 disminuye la probabilidad de tener Diabetes.

### 5.5 Partial Dependence Plot (4 puntos)

Finalmente, se le pide generar un gráfico del tipo Partial Dependence Plot para las mismas 5 variables con mayor impacto global usando una submuestra de 1000 observaciones. ¿Qué relación existe entre la salida promedio del modelo y cada feature analizada? ¿Son estas conclusiones generalizables para todo el conjunto de datos?

In [ ]:
# Inserte código para generar gráficos PDP aquí
submuestra = shap.utils.sample(preprocessor.transform(X_test), 1000)
for feature in top5:
  shap.plots.partial_dependence(feature, pipeline.named_steps['classifier'].predict, submuestra)
  plt.show()

Respuestas:

Los resultados presentes corresponden a los mismos concluidos en la sección anterior, mayor Age, un mayor HighChol, mayor HighBP y mayor BMI la probabilidad de tener Diabetes aumenta, mientras que si aumenta GenHlth_2.0 disminuye la probabilidad de tener Diabetes.

Ya que la muestra utilizada es muy pequeña en comparación a el tamaño de la base de datos, no es recomendable generalizar los resultados obtenidos.

## 6. Sistema de Reglas! (10 punto)

<p align="center">
  <img src="https://media.baamboozle.com/uploads/images/125978/1638281150_1380186_gif-url.gif" width="400">
</p>

Después de todo el trabajo hecho, Dr. Simi le pide simplificar el funcionamiento de su modelo en un sistema de reglas que le permita explicar a sus clientes las predicciones que genera su modelo.
En particular, Dr. Simi le pide explicar la decisión tomada para las observaciones **1000**, **3001** y **5751**. Con las reglas propuestas señale a **cuánta población** es posible explicar con estas reglas e indique la **precisión** que poseen las reglas en la totalidad de los datos. ¿Tienen sentido sus reglas propuestas para las observaciones?. Fundamente sus respuesta señalando el impacto que tienen sus reglas sobre todo el conjunto de datos.

`Hint:` Como debe entregar las columnas que entran al clasificador entrenado de su pipeline, le será útil extraer el paso de preprocesamiento y generar dataframes preprocesados para el conjunto `train` y `test`.

In [ ]:
!pip install alibi

In [ ]:
from alibi.explainers import AnchorTabular

In [ ]:
# Se crean los datos según el hint
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Se entrena el modelo
model = XGBClassifier()
model.fit(X_train_preprocessed, y_train)

In [ ]:
# Inserte su código para generar sistema de reglas aquí
feature_names = X_train_preprocessed.columns.tolist()
explicador = AnchorTabular(model.predict, feature_names=feature_names, seed= 444)
explicador.fit(X_train_preprocessed.values)

idx = 1000
# explicar predicción idx 1000
explicacion1000 = explicador.explain(X_train_preprocessed.iloc[idx].values)
print('Explicación para observación de 1000')
print('Reglas aplicadas: ' + str(explicacion1000.anchor))
print('Cobertura de población estimada: ' + str(explicacion1000.coverage))
print('Precisión obtenida: ' + str(explicacion1000.precision))

print('--------------------------------------------------------------------------')

idx = 3001
# explicar predicción idx 3001
explicacion3001 = explicador.explain(X_train_preprocessed.iloc[idx].values)
print('Explicación para observación de 3001')
print('Reglas aplicadas: ' + str(explicacion3001.anchor))
print('Cobertura de población estimada: ' + str(explicacion3001.coverage))
print('Precisión obtenida: ' + str(explicacion3001.precision))

print('--------------------------------------------------------------------------')

idx = 5751
# explicar predicción idx 5751
explicacion5751 = explicador.explain(X_train_preprocessed.iloc[idx].values)
print('Explicación para observación de 5751')
print('Reglas aplicadas: ' + str(explicacion5751.anchor))
print('Cobertura de población estimada: ' + str(explicacion5751.coverage))
print('Precisión obtenida: ' + str(explicacion5751.precision))

Respuestas:

Observando los resultados obtenidos se puede observar que las reglas se contradicen entre si, ya que en algunos puntos pide cierto valor para una característica para que sea considerado Diabetes mientras que en otra pide el rango inverso de esta. Además se puede observar que se tiene una cobertura muy baja de la población para cada regla, sin embargo una precisión extremadamente alta, por lo que se concluye que estas reglas serían consideradas para un subconjunto de la población muy específico donde estas reglas funcionaran.

# Retrospectiva... (10 puntos)

En base a los diferentes métodos que implementa y ha comentado en este laboratorio, comente qué métodos le permiten entregar mejores conclusiones para la tarea de clasificación de diabetes. Por otro lado, ¿qué métodos son más útiles para el problema del doctor Simi, métodos agnosticos locales o globales?

Cada uno de los métodos tratados en este laboratorio tiene un enfoque distinto, por lo que la elección del más adecuado dependerá de los objetivos que se busquen satisfacer, considerando algunas limitaciones, como serían los costos asociados al procesamiento de datos, la generación de modelos o el nivel de detalle requerido para las características.

Por otro lado, la utilidad de un método para el doctor Simi dependerá del tipo de información que desea obtener. Si busca información general o busca tomar decisiones de manera amplia, por lo que se recomiendaría utilizar métodos globales. En cambio, si el objetivo es abordar el problema de forma más específica para un individu en específico, se debería optar por los métodos locales.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://media.tenor.com/ctnq4bhZE3gAAAAC/drsimi-simi.gif)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>